In [4]:
import numpy as np
import pandas as pd
from surprise import Reader, Dataset
from surprise import SVD
from surprise import SVD, NMF, KNNWithMeans, CoClustering
import surprise
from surprise.model_selection import train_test_split
from collections import defaultdict
from surprise import Dataset
from surprise import SVD
from surprise.model_selection import KFold

In [7]:
data = pd.read_csv('dataset_movie_100k.csv')[['user_id','book_id','rating']]

In [8]:
print("Le dataset contient ", data.shape[0], " notes.")

data.head()

Le dataset contient  100004  notes.


,user_id,book_id,rating
0,68,1,4.0
1,261,1,1.5
2,383,1,5.0
3,262,1,2.5
4,77,1,4.0


In [9]:
reader = Reader()
data = Dataset.load_from_df(data, reader)

trainset, testset = train_test_split(data, test_size=.25)

In [10]:
algo = SVD(n_factors=10, n_epochs=10,lr_all=.01, reg_all=.01)
#algo= NMF(10)
#algo=KNNWithMeans(epochs=10)
#algo=CoClustering()

## precison et recall

In [11]:
#https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-compute-precision-k-and-recall-k

In [12]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls


def F_score(pr,rc):
    return 2*(pr*rc)/(rc+pr)

### K-fold (ici 5-fold)

In [13]:
kf = KFold(n_splits=5)
indice_fold = 1

res_precision = []
res_recall= []
f_score=[]

for trainset, testset in kf.split(data):
    print('='*50)
    print ('FOLD # ' + str(indice_fold))
    print('='*50)
    
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=4)

    avg_precision = sum(prec for prec in precisions.values()) / len(precisions)
    avg_recall = sum(rec for rec in recalls.values()) / len(recalls)
    f=F_score(avg_precision,avg_recall)
    
    print (avg_precision)
    print (avg_recall)
    print(f)
    
    res_precision.append(avg_precision)
    res_recall.append(avg_recall)
    f_score.append(f)
    indice_fold += 1
    
    
print('='*50)
print ('MOYENNES')
print('='*50)
print('précision@k moyenne: ' + str(np.mean(res_precision)))
print('rappel@k moyen: ' + str(np.mean(res_recall)))
print('F-score moyen: ' + str(np.mean(f_score)))


FOLD # 1
0.8479922172545122
0.3078677527667666
0.45173198321773916
FOLD # 2
0.8577523098791761
0.3200617680542783
0.4661749693708131
FOLD # 3
0.8654755973141622
0.3228752807659933
0.4702999452996194
FOLD # 4
0.8638125990585005
0.31420917544904997
0.460802763354919
FOLD # 5
0.8553778725420514
0.31506903979532624
0.4605131289992411
MOYENNES
précision@k moyenne: 0.8580821192096805
rappel@k moyen: 0.3160166033662829
F-score moyen: 0.46190455804846636
